# LungsNet: Model Training
Training DenseNet121 for Pneumonia Detection.

In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os

## 1. Data Loading & Augmentation
Note: The training dataset has been augmented offline to balance the classes.

In [16]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
# Paths
TRAIN_DIR = "../data/balanced/train"
TEST_DIR = "../data/raw/chest_xray/test"
# Note: We use the balanced TRAIN_DIR for both train and validation split

# Add validation_split=0.2 to create a 20% validation set from training data
train_val_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% for validation
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Training Generator (Subset: Training)
train_generator = train_val_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

# Validation Generator (Subset: Validation, uses same TRAIN_DIR)
val_generator = train_val_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

# Test Generator (Independent Test Set)
test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)


Found 6214 images belonging to 2 classes.
Found 1553 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


## 2. Build Model (DenseNet121)

In [17]:
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
# for layer in base_model.layers:
#     layer.trainable = False
# Or fine-tune (optional)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

## 3. Train Model

In [18]:
EPOCHS = 10

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_steps=val_generator.samples // BATCH_SIZE
)

Epoch 1/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 849s 4s/step - accuracy: 0.9478 - loss: 0.1306 - val_accuracy: 0.6602 - val_loss: 1.0285
Epoch 2/10
  1/194 ━━━━━━━━━━━━━━━━━━━━ 12:04 4s/step - accuracy: 1.0000 - loss: 0.0132

d:\Tugas kuliah\SEM 5\UAS\BigDataMining\LungsNet\venv\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


194/194 ━━━━━━━━━━━━━━━━━━━━ 49s 232ms/step - accuracy: 1.0000 - loss: 0.0132 - val_accuracy: 0.7135 - val_loss: 0.8131
Epoch 3/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 760s 4s/step - accuracy: 0.9756 - loss: 0.0683 - val_accuracy: 0.9486 - val_loss: 0.1411
Epoch 4/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 47s 224ms/step - accuracy: 0.9688 - loss: 0.0709 - val_accuracy: 0.9564 - val_loss: 0.1218
Epoch 5/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 754s 4s/step - accuracy: 0.9843 - loss: 0.0472 - val_accuracy: 0.7148 - val_loss: 0.9387
Epoch 6/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 48s 229ms/step - accuracy: 0.9688 - loss: 0.0643 - val_accuracy: 0.6771 - val_loss: 1.1176
Epoch 7/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 773s 4s/step - accuracy: 0.9874 - loss: 0.0356 - val_accuracy: 0.9733 - val_loss: 0.0620
Epoch 8/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 49s 227ms/step - accuracy: 1.0000 - loss: 0.0346 - val_accuracy: 0.9818 - val_loss: 0.0565
Epoch 9/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 759s 4s/step - accuracy: 0.9901 - loss: 0.0277 - val_accurac

## 4. Save Model

In [19]:
model.save('../models/lungsnet_densenet121.h5')
print("Model saved successfully!")

Model saved successfully!
